In [346]:
import requests
import time
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import json
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from math import ceil, floor
from selenium.webdriver.common.keys import Keys

In [9]:
url = "https://api.wazirx.com/api/v2/tickers"

In [436]:
bases = ['btc', 'wrx', 'usdt']

In [426]:
# bases = ['usdt']

In [344]:
og = ['btc', 'wrx', 'usdt', 'inr']

In [376]:
precision = None
with open("./Precision.json", "r") as _f:
    precision = json.load(_f)

In [252]:
quotes = {
    'inr' : set(),
    'btc' : set(),
    'usdt' : set(),
    'wrx' : set()
}

In [241]:
market = requests.get(url).json()

In [ ]:
t = 0
flag = True
while(True):
    global market
    if t%60 == 0:
        print("Time :", t)
    market = requests.get(url).json()
    
    quotes = {
        'inr' : set(),
        'btc' : set(),
        'usdt' : set(),
        'wrx' : set()
    }
    
    for pair in market.values():
        quotes[pair['quote_unit']].add(pair['base_unit'])
    
    for base in bases:
        flag = True
        for cur in quotes[base]:
            if cur in quotes['inr'] and precision['inr'][cur] == 0:
                try:
                    capital = 10000
                    capital = capital/float(market[base+'inr']['sell'])
                    capital = capital/float(market[cur+base]['sell'])
                    capital = capital*(float(market[cur+'inr']['buy']))
    #                 print(capital)
                    if capital > 10100:
                        print('inr -> '+base+' -> '+cur+' -> '+'inr :', capital)
                        place_triangular_arbitrage('inr', base, cur, market)
                        flag=False
                        break
                except ZeroDivisionError:
                    pass
        if not flag:
            break
    #                 print("Zero Error On : "+'inr -> '+base+' -> '+cur+' -> '+'inr :')
    time.sleep(5)
    t+=5

Time : 0
Time : 60
inr -> usdt -> dent -> inr : 10128.55121022622
Precision : 2 0
Quantity : ['12.21', '1507', '1507']
Prices :  ['81.91', '0.0081', '0.672']
 ### Result ### 
Input Capital : 1000.12
Output Capital : 1012.704
Profit : 1.2582490101187804
 ###  Done  ###
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Hi
Time : 120
Time : 180
Time : 240
Time : 300
Time : 360
Time : 420
Time : 480
Time : 540
Time : 600
Time : 660
Time : 720
Time : 780
Time : 840
Time : 900


In [458]:
def place_triangular_arbitrage(fiat, base, cur, market):
    
    quantity = [0, 0, 0]
#     prices = [market[base+fiat]['sell'], market[cur+base]['sell'], market[cur+fiat]['buy'][:-1]+str((int(market[cur+fiat]['buy'][-1])-1)%10)]
    prices = [market[base+fiat]['sell'], market[cur+base]['sell'], market[cur+fiat]['buy']]
    cur_precision = min(get_precision(base, cur), get_precision(fiat, cur))
    base_precision = get_precision(fiat, base)
    
    print("Precision :", base_precision, cur_precision)
    capital = 1000
    quantity[0] = floor(capital*(10**(base_precision))/float(market[base+fiat]['sell']))/(10**(base_precision))
    quantity[1] = ceil(quantity[0]*(10**(cur_precision))/float(market[cur+base]['sell']))/(10**(cur_precision))
    quantity[2] = quantity[1]
    
    quantity[0] = ceil((quantity[1]*float(market[cur+base]['sell']))*(10**base_precision))/(10**base_precision)
    
    if cur_precision == 0:
        quantity[1] = int(quantity[1])
        quantity[2] = int(quantity[2])
    if base_precision == 0:
        quantity[0] = int(quantity[1])
    
    quantity = [str(x) for x in quantity]
    print("Quantity :", quantity)
    print("Prices : ",prices)
    
    input_precision = get_price_precision(market[base+fiat])
    output_precision = get_price_precision(market[cur+fiat])
    input_cap = floor(float(quantity[0])*float(prices[0])*(10**input_precision))/(10**input_precision)
    output_cap = floor(float(quantity[2])*float(prices[2])*(10**output_precision))/(10**output_precision)
    
    print(" ### Result ### ")
    print("Input Capital :", input_cap)
    print("Output Capital :", output_cap)
    print("Profit :", (output_cap-input_cap)*100/(input_cap))
    print(" ###  Done  ###")
    
    
#     time.sleep(1)
    place_order(base, cur, prices[1], 'Buy', quantity[1])
    place_order(fiat, base, prices[0], 'Buy', quantity[0])
    place_order(fiat, cur, prices[2], 'Sell', quantity[2])

#     capital = quantity[0]*price[0]*(10**)
        

In [378]:
def get_precision(base, coin):
    return precision[base][coin]

In [403]:
def get_price_precision(ticker):
    return max(
        0 if len(ticker['low'].split('.')) == 1 else len(ticker['low'].split('.')[1]),
        0 if len(ticker['high'].split('.')) == 1 else len(ticker['high'].split('.')[1]),
        0 if len(ticker['last'].split('.')) == 1 else len(ticker['last'].split('.')[1]),
        0 if len(str(ticker['open']).split('.')) == 1 else len(str(ticker['open']).split('.')[1]),
        0 if len(ticker['buy'].split('.')) == 1 else len(ticker['buy'].split('.')[1]),
        0 if len(ticker['sell'].split('.')) == 1 else len(ticker['sell'].split('.')[1])
    )

In [390]:
place_triangular_arbitrage('inr', 'usdt', 'xlm', market)

Precision : 2 1
Quantity : ['12.21', '29.0', '29.0']
Prices :  ['81.97', '0.42096', '34.9484']


In [71]:
def get_browser():
#     tor_proxy = "127.0.0.1:9150"
    chromeoptions = webdriver.ChromeOptions()
    chromeoptions.add_argument("--incognito")
    chromeoptions.add_argument('--no-sandbox')
    chromeoptions.add_argument('--window-size=1920,1080')
    chromeoptions.add_argument('--disable-gpu')
    chromeoptions.add_argument('--disable-dev-shm-usage')
    chromeoptions.add_argument('--ignore-certificate-errors')
#     chromeoptions.add_argument('--headless')
#     chromeoptions.add_argument('--proxy-server=socks5://%s' % tor_proxy)

#     browser = webdriver.Chrome('C:\\Users\\Siddak\\Documents\\CloudSEK\\chromedriver', options=chromeoptions)
    browser = webdriver.Chrome(ChromeDriverManager().install(), options=chromeoptions)
    return browser

In [460]:
browser = get_browser()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/siddakbakshi/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [461]:
browser.get("https://wazirx.com/")

In [468]:
def place_order(base, coin, price, typ, qty):
    idx = 0
    if base == 'usdt':
        idx = 17
    base_button = browser.find_elements_by_xpath("//*[ text() = '"+base+"' ]")[idx]
    base_button.click()
    coin_search = browser.find_element_by_class_name("currency-search")
    x = 6
    while(x):
        coin_search.send_keys(Keys.BACKSPACE)
        x-=1
    coin_search.send_keys(coin)
    coin_select = browser.find_element_by_class_name("ticker-item")
    coin_select.click()
    order_type = browser.find_element_by_xpath("//*[ text() = '"+typ+"' ]")
    order_type.click()
    x = 6
    while(x):
        coin_search.send_keys(Keys.BACKSPACE)
        x-=1
    price_input = browser.find_elements_by_tag_name("input")[-3]
    x = 10
    while(x):
        price_input.send_keys(Keys.BACKSPACE)
        x-=1
    price_input.send_keys(str(price))
    quantity_input = browser.find_elements_by_tag_name("input")[-2]
    x = 10
    while(x):
        quantity_input.send_keys(Keys.BACKSPACE)
        x-=1
    quantity_input.send_keys(str(qty))
    flag = True
    while(flag):
        place_ordr = browser.find_elements_by_tag_name("button")[-1]
        place_ordr.click()
        while(True):
            try:
                time.sleep(0.3)
                notifications = browser.find_elements_by_class_name("notification")[-1]
                break
            except IndexError:
                continue
        if "Order placed successfully" not in notifications.text:
            time.sleep(1)
        else:
            flag = False

In [470]:
place_order('inr', 'usdt', '85.0', 'Sell', '1')

Hi


In [253]:
for pair in market.values():
    quotes[pair['quote_unit']].add(pair['base_unit'])

In [259]:
currencies = set()

In [260]:
currencies = currencies.union(quotes['inr']).union(quotes['usdt']).union(quotes['wrx']).union(quotes['btc'])

In [264]:
for c in currencies:
    quantity_p = -1
    for b in bases+['inr']:
        if c+b in market:
            if quantity_p < 0  or len(market[c+b]['volume'].split('.')[1]) == quantity_p:
                quantity_p = len(market[c+b]['volume'].split('.')[1])
            else:
                print("Pricision Not Matched for : ", c)

Pricision Not Matched for :  eos
Pricision Not Matched for :  ftt
Pricision Not Matched for :  ftt
Pricision Not Matched for :  bat
Pricision Not Matched for :  luna
Pricision Not Matched for :  nano
Pricision Not Matched for :  avax
Pricision Not Matched for :  icx
Pricision Not Matched for :  zec
Pricision Not Matched for :  eth
Pricision Not Matched for :  dot
Pricision Not Matched for :  atom
Pricision Not Matched for :  atom
Pricision Not Matched for :  bnb
Pricision Not Matched for :  bnb
Pricision Not Matched for :  easy
Pricision Not Matched for :  link
Pricision Not Matched for :  pax
Pricision Not Matched for :  usdc
